# Introduction

The NYC 311 Service Requests and Community District Indicators have both been obtained and exported. In this section, we'll merge the datasets on the community district/board columns, deal with missing values, reformat values for consistency, and exclude superfluous columns. 

# Loading Dependencies

In [1]:
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Data Cleaning

### Dealing with Missing and Superfluous Data

Below, we load the 311 data from Google Drive using [GoogleAuth](https://pythonhosted.org/PyDrive/oauth.html) and read it into a pandas dataframe.

In [7]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [9]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [10]:
path = '/content/drive/MyDrive/Colab Notebooks/311.csv'
df = pd.read_csv(path, index_col=0)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (39,45,46) have mixed types.Specify dtype option on import or set low_memory=False.

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



Let's now check the column data types and identify columns with missing values.

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1500000 entries, 0 to 1499999
Data columns (total 46 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   unique_key                      1500000 non-null  int64  
 1   created_date                    1500000 non-null  object 
 2   agency                          1500000 non-null  object 
 3   agency_name                     1500000 non-null  object 
 4   complaint_type                  1500000 non-null  object 
 5   descriptor                      1422438 non-null  object 
 6   location_type                   1237278 non-null  object 
 7   incident_zip                    1467586 non-null  float64
 8   incident_address                1437587 non-null  object 
 9   street_name                     1437527 non-null  object 
 10  cross_street_1                  1217249 non-null  object 
 11  cross_street_2                  1216469 non-null  object 
 12  

The dataset consists of float, integer, and object data types. Most of the columns that contain missing values appear to be associated with particular complaint types. Next, we overwrite the dataframe to include only the rows and columns of interest.

In [12]:
#eliminate rows without lat/long and location type data
df = df[df.latitude.notnull()]
df = df[df.longitude.notnull()]
df = df[df.descriptor.notnull()]

In [13]:
#exclude unnecessary columns
df = df[['created_date', 'complaint_type', 'agency', 'community_board', 
         'latitude', 'longitude', 'descriptor', 'borough']]


### Descriptions of Remaining Columns

The descriptions for every column are available on the dataset's NYCOpenData profile. The descriptions for the columns included in this analysis have been pasted below.
*   **Created Date**: Date service request(SR) was created
*   **Complaint Type**: This is the first level of a hierarchy identifying the topic of the incident or condition. Complaint Type may have a corresponding Descriptor (below) or may stand alone.
*   **Agency**: Acronym of responding City Government Agency
*   **Community Board**: Provided by geovalidation.
*   **Latitude**: Geo based Lat of the incident location
*   **Longitude**: eo based Long of the incident location
*   **Descriptor**: This is associated to the Complaint Type, and provides further detail on the incident or condition. Descriptor values are dependent on the Complaint Type, and are not always required in SR.
*   **Borough**: Provided by the submitter and confirmed by geovalidation.




### Reformatting Values for Consistency

If we look at unique values in the 'complaint type' column, we'll see that some values are in title case while others are in upper case.  

In [14]:
df.complaint_type.unique()

array(['Noise - Residential', 'Illegal Parking',
       'Noise - Street/Sidewalk', 'NonCompliance with Phased Reopening',
       'Blocked Driveway', 'Noise - Vehicle', 'Street Sign - Damaged',
       'Overgrown Tree/Branches', 'Animal-Abuse', 'Noise - Commercial',
       'Mass Gathering Complaint', 'Traffic Signal Condition',
       'Non-Emergency Police Matter', 'Rodent', 'Street Sign - Missing',
       'Noise - Park', 'Maintenance or Facility', 'Illegal Tree Damage',
       'Food Establishment', 'Street Condition', 'Street Light Condition',
       'Noise', 'HEAT/HOT WATER', 'UNSANITARY CONDITION',
       'Construction Lead Dust', 'GENERAL', 'Air Quality', 'ELECTRIC',
       'DOOR/WINDOW', 'PLUMBING', 'APPLIANCE',
       'Unsanitary Animal Facility', 'Drinking', 'Sidewalk Condition',
       'PAINT/PLASTER', 'Traffic', 'Water System', 'Curb Condition',
       'Indoor Sewage', 'WATER LEAK', 'Sewer', 'Outdoor Dining',
       'FLOORING/STAIRS', 'Drug Activity', 'Violation of Park Rules',


Below, all of the values in the column are converted to title case.

In [15]:
df.complaint_type = df.complaint_type.str.title()

The values in the 'agengy' column also have inconsistent formatting: all but one have abbreviated names.

In [16]:
df.agency.unique()

array(['NYPD', 'MAYORâ\x80\x99S OFFICE OF SPECIAL ENFORCEMENT', 'DOT',
       'DPR', 'DOHMH', 'DEP', 'HPD', 'EDC', 'TLC', 'DCA', 'DOE', 'DOB',
       'DSNY', 'DOITT'], dtype=object)

We'll convert the ''MAYOR'S OFFICE OF SPECIAL ENFORCEMENT'value to its abbreviated name, OSE:


In [17]:
def update_OSE(agency):

  """Takes in an agency name and converts to abbreviated form, if the agency is
     the Office of Special Enforcement."""

  if agency == 'MAYORâ\x80\x99S OFFICE OF SPECIAL ENFORCEMENT':
      return "OSE"
  else:
      return agency
    
df.agency = df.agency.map(lambda agency: update_OSE(agency))

Among noise-related complaints, only one does not specify the noise source:

In [18]:
noise_columns = []

for i in df.complaint_type:
   if i.startswith('Noise') and i not in noise_columns:
     noise_columns.append(i)
noise_columns

['Noise - Residential',
 'Noise - Street/Sidewalk',
 'Noise - Vehicle',
 'Noise - Commercial',
 'Noise - Park',
 'Noise',
 'Noise - Helicopter',
 'Noise - House Of Worship']

We'll convert the generic 'Noise' value to 'Noise - Unspecified' so that its formatting matches that of similar complaint types.

In [19]:
#Label unspecified noise complaints as 'Noise - Unspecified'

def update_noise(complaint):

  """Takes in an complaint name and updates it, if it is
     an unspecified noise complaint."""

  if complaint == 'Noise':
    
    return "Noise - Unspecified"

  else:

    return complaint
    
df.complaint_type = df.complaint_type.map(
    lambda complaint: update_noise(complaint))

The 'created_date' column currently contains string values:


In [20]:
type(df.created_date[1])

str

The ```pandas.to_datetime()``` method converts each string value to a datetime object below.

In [21]:
#convert date column to datetime 
df['created_date'] = pd.to_datetime(df['created_date'], 
                                    format='%Y-%m-%dT%H:%M:%S.%f')

In [22]:
#check new data type for created_date column
type(df['created_date'][1])

pandas._libs.tslibs.timestamps.Timestamp

Using the datetime format, we can now create new columns with the day, month, day of week, and hour of each call. This will allow us to explore the independent relationship of each variable to other characteristics of the calls.

In [23]:
df['day'] = [str(i.date()) for i in df.created_date]
df['month'] = [int(i.month) for i in df.created_date]
df['day_of_week'] = [int(i.weekday()) for i in df.created_date]
df['hour'] = [int(i.hour) for i in df.created_date]

The calls are currently ordered in the dataframe from most-least recent. For data visualization purposes, we'll reindex the calls so that the linear, left-to-right presentation of time on an x-axis runs from least-most recent.

In [24]:
#reorder df from least-most recent calls
df = df.reindex(index=df.index[::-1])

To numerically encode the descriptor column in the modeling phase, we'll first need to exclude all values that aren't letters. Below, an empty space is substituted for all numbers and non-letter symbols.

In [25]:
num_pattern = r'[-()0-9]'
df.descriptor = df.descriptor.map(lambda x: re.sub(num_pattern, '', x).lower())
df.descriptor = df.descriptor.map(lambda x: re.sub('/', ' ', x))

In [26]:
#preview sample of unique descriptors with non-letters removed
df.descriptor.unique()[0:20]

array(['construction', 'no access', 'blocked hydrant',
       'posted parking sign violation', ' derelict vehicles',
       'manhole cover missing emergency sa',
       'noise: construction before after hours nm', 'banging pounding',
       'loud talking', 'social distancing', 'leak use comments wa',
       'noise: private carting noise nq', 'engine idling',
       'e dirty sidewalk', 'rodent sighting',
       'possible water main break use comments wa', 'loud music party',
       'rat sighting', 'dumpster  construction waste', 'dirty water we'],
      dtype=object)

### Merging the Community District Indicators Data

Now let's read in and preview the community district indicators data.

In [27]:
path = '//content/drive/MyDrive/Colab Notebooks/indicators.csv'
ind_df = pd.read_csv(path)

In [28]:
ind_df.head()

,the_geom,cartodb_id,the_geom_webmercator,acres,acs_tooltip,acs_tooltip_2,acs_tooltip_3,area_sqmi,borocd,cb_email,cb_website,cd_full_title,cd_short_title,cd_son_fy2018,cd_tot_bldgs,cd_tot_resunits,count_hosp_clinic,count_libraries,count_parks,count_public_schools,crime_count,crime_count_boro,crime_count_nyc,crime_per_1000,crime_per_1000_boro,crime_per_1000_nyc,female_10_14,female_15_19,female_20_24,female_25_29,female_30_34,female_35_39,female_40_44,female_45_49,female_5_9,female_50_54,female_55_59,female_60_64,female_65_69,female_70_74,...,pct_asian_nh,pct_bach_deg,pct_bach_deg_boro,pct_bach_deg_nyc,pct_black_nh,pct_clean_strts,pct_clean_strts_boro,pct_clean_strts_nyc,pct_foreign_born,pct_hh_rent_burd,pct_hh_rent_burd_boro,pct_hh_rent_burd_nyc,pct_hispanic,pct_other_nh,pct_served_parks,pct_white_nh,pop_2000,pop_2010,pop_acs,pop_change_00_10,poverty_rate,poverty_rate_boro,poverty_rate_nyc,puma,shared_puma,shared_puma_cd,son_issue_1,son_issue_2,son_issue_3,total_lot_area,under18_rate,under18_rate_boro,under18_rate_nyc,unemployment_boro,unemployment,unemployment_nyc,v_pluto,v_acs,v_facdb,v_crime
0,NaN,1,NaN,976.3,American Community Survey 2014-2018 5-Year Est...,American Community Survey (ACS) 2013-2017 5-ye...,2010 Census population counts for floodplain a...,1.5,101,man01@cb.nyc.gov,www1.nyc.gov/site/manhattancb1/index.page,Manhattan Community District 1,Manhattan CD 1,NaN,1725,45629,10,2,13,19,1055,26271,92480,6.8,16.1,11,1.1,2.4,3.9,7.2,6.5,4.7,3.6,2.9,1.8,2.3,2.4,2.6,2.4,1.8,...,15.5,82.2,60.8,37.4,2.2,95.4,94.1,95.1,23.3,30.1,36.4,44.2,6.5,3.6,100,72.2,34420,60978,154636,0.77,8.8,14.4,19.8,3810,True,Manhattan CD 2,Infrastructure resiliency,Traffic,Other,2881166.373,12.7,14.4,20.9,3.8,2.8,4.4,20v4,Y2014-2018,6/24/20,2019
1,NaN,2,NaN,865.9,American Community Survey 2014-2018 5-Year Est...,American Community Survey (ACS) 2013-2017 5-ye...,2010 Census population counts for floodplain a...,1.4,102,bgormley@cb.nyc.gov,www.nyc.gov/html/mancb2,Manhattan Community District 2,Manhattan CD 2,NaN,5256,60610,19,3,6,8,2313,26271,92480,15.0,16.1,11,1.1,2.4,3.9,7.2,6.5,4.7,3.6,2.9,1.8,2.3,2.4,2.6,2.4,1.8,...,15.5,82.2,60.8,37.4,2.2,96.3,94.1,95.1,23.3,30.1,36.4,44.2,6.5,3.6,100,72.2,93119,90016,154636,-0.03,8.8,14.4,19.8,3810,True,Manhattan CD 1,Parks,Schools,Senior services,2231472.120,12.7,14.4,20.9,3.8,2.8,4.4,20v4,Y2014-2018,6/24/20,2019
2,NaN,3,NaN,1076.9,American Community Survey 2014-2018 5-Year Est...,American Community Survey (ACS) 2013-2017 5-ye...,2010 Census population counts for floodplain a...,1.7,103,mn03@cb.nyc.gov,www.nyc.gov/html/mancb3,Manhattan Community District 3,Manhattan CD 3,NaN,4644,82557,41,5,13,42,2360,26271,92480,15.2,16.1,11,1.5,2.9,4.2,6.4,4.1,3.2,3.0,3.2,1.2,2.8,3.3,3.4,2.9,2.1,...,30.6,43.3,60.8,37.4,8.4,91.0,94.1,95.1,34.5,38.4,36.4,44.2,25.1,2.6,100,33.3,164407,163277,154995,-0.01,19.3,14.4,19.8,3809,False,NaN,Affordable housing,Senior services,Homelessness,2985434.526,11.4,14.4,20.9,3.8,3.6,4.4,20v4,Y2014-2018,6/24/20,2019
3,NaN,4,NaN,1131.6,American Community Survey 2014-2018 5-Year Est...,American Community Survey (ACS) 2013-2017 5-ye...,2010 Census population counts for floodplain a...,1.8,104,jbodine@cb.nyc.gov,www.nyc.gov/mcb4,Manhattan Community District 4,Manhattan CD 4,NaN,3829,85706,38,2,7,29,2792,26271,92480,18.1,16.1,11,0.9,1.7,4.0,7.2,6.4,3.8,3.0,2.9,1.0,2.3,2.7,2.7,2.1,2.1,...,17.2,74.1,60.8,37.4,5.2,95.6,94.1,95.1,30.8,33.9,36.4,44.2,15.5,3.0,94,59.1,87479,103245,154496,0.18,11.3,14.4,19.8,3807,True,Manhattan CD 5,Affordable housing,"Land use trends (zoning, development, neighbor...",Traffic,3110433.364,8.2,14.4,20.9,3.8,3.5,4.4,20v4,Y2014-2018,6/24/20,2019
4,NaN,5,NaN,1005.4,American Community Survey 2014-2018 5-Year Est...,American Community Survey (ACS) 2013-2017 5-ye...,2010 Census population counts for floodplain a...,1.6,105,office@cb5.org,www.cb5.org,Manhattan Community District 5,Manhattan CD 5,NaN,3127,49436,23,7,3,13,4538,26271,92480,29.4,16.1,11,0.9,1.7,4.0,7.2,6.4,3.8,3.0,2.9,1

In [29]:
ind_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Columns: 188 entries, the_geom to v_crime
dtypes: bool(1), float64(121), int64(51), object(15)
memory usage: 86.4+ KB


This dataset contains many columns, which have been narrowed down to a selection of development, demographic, economic and social variables below. Most of the column names are self-explanatory, but a a full list of column descriptions can be found in the 'Indicators Data Dictionary' file on any [community district profile](https://communityprofiles.planning.nyc.gov/).

In [30]:
ind_df = ind_df[[
'cd_short_title', 
'area_sqmi',
'count_hosp_clinic',
'count_libraries',
'count_parks',
'count_public_schools',
'crime_per_1000',
'female_15_19',
'female_20_24',
'female_25_29',
'female_30_34',
'female_35_39',
'female_40_44',
'female_45_49',
'female_50_54',
'female_55_59',
'female_60_64',
'male_10_14',
'male_15_19',
'male_20_24',
'male_25_29',
'male_30_34',
'male_35_39',
'male_40_44',
'male_45_49',
'male_50_54',
'male_55_59',
'male_60_64',
'under18_rate',
'over65_rate',
'lots_total',
'lots_vacant',
'pct_white_nh',
'pct_black_nh',
'pct_asian_nh',
'pct_other_nh',
'pct_hispanic',
'mean_commute',
'pop_change_00_10',
'pct_clean_strts',
'unemployment',
'poverty_rate',
'lep_rate_boro',
'pct_hh_rent_burd'
]]

Let's look at the summary statistics for each of the numeric variables in the dataset.

In [31]:
ind_df.describe().loc[['mean','std', 'min', 'max']]

,area_sqmi,count_hosp_clinic,count_libraries,count_parks,count_public_schools,crime_per_1000,female_15_19,female_20_24,female_25_29,female_30_34,female_35_39,female_40_44,female_45_49,female_50_54,female_55_59,female_60_64,male_10_14,male_15_19,male_20_24,male_25_29,male_30_34,male_35_39,male_40_44,male_45_49,male_50_54,male_55_59,male_60_64,under18_rate,over65_rate,lots_total,lots_vacant,pct_white_nh,pct_black_nh,pct_asian_nh,pct_other_nh,pct_hispanic,mean_commute,pop_change_00_10,pct_clean_strts,unemployment,poverty_rate,lep_rate_boro,pct_hh_rent_burd
mean,4.613559,22.898305,3.677966,6.915254,25.694915,10.205085,2.720339,3.540678,4.883051,4.398305,3.757627,3.313559,3.308475,3.318644,3.172881,2.988136,2.794915,2.720339,3.342373,4.601695,4.237288,3.538983,3.159322,3.083051,3.045763,2.861017,2.513559,20.872881,13.832203,14531.593220,458.000000,32.264407,21.822034,13.088136,2.927119,29.898305,41.238983,0.038475,94.064407,4.513559,19.940678,23.049153,44.723729
std,4.295164,13.101323,1.665478,4.035567,8.987068,4.627594,0.683243,0.844887,1.364848,1.113629,0.547787,0.368809,0.402273,0.563081,0.541973,0.529178,0.797285,0.788447,0.844904,1.219453,1.238450,0.827338,0.655249,0.448043,0.453463,0.467944,0.480466,5.383334,3.643816,11987.065263,581.954495,24.355979,22.333160,12.294884,2.074129,20.544053,6.309572,0.111801,3.076146,1.465518,6.824982,5.305637,7.550322
min,1.400000,1.000000,1.000000,1.000000,8.000000,2.600000,1.100000,1.800000,2.900000,3.000000,2.900000,2.600000,2.100000,1.800000,2.000000,2.000000,0.800000,1.100000,1.200000,2.700000,2.500000,2.500000,1.800000,2.100000,2.100000,2.000000,1.400000,8.200000,8.200000,1461.000000,25.000000,1.100000,1.000000,0.700000,1.200000,6.500000,25.400000,-0.100000,85.100000,2.000000,7.200000,10.900000,28.800000
max,21.500000,54.000000,9.000000,20.000000,47.000000,29.400000,4.200000,6.900000,8.400000,6.900000,5.600000,4.300000,4.000000,4.400000,4.200000,3.900000,4.800000,4.200000,5.600000,8.100000,7.300000,5.600000,5.300000,4.200000,4.000000,3.800000,3.300000,36.800000,23.900000,50434.000000,3364.000000,82.900000,86.600000,52.700000,14.900000,71.500000,51.100000,0.770000,99.100000,7.800000,35.600000,28.800000,60.300000


We can see that the average size of a community district is 4.6 miles, and that the average percentage of men and women in their 20s and 30s are higher than percentages for other age groups. On average, white residents make up about a third of the community districts' population, followed by hispanic, black, and asian residents. The average poverty rate is 19%, the average unemployment rate is 4.5%, and the average crime rate (for 7 major felonies) per 1000 residents is about 10. 

We'll format the community district column in this dataset such that it can be merged with the community board column in the 311 dataset. In NYC, community districts and their corresponding boards share the same number and borough. According to the webpage for [Queens Community Board 3](http://www.cb3qn.nyc.gov/1652), "each community district has a Community Board to oversee the delivery of city services and facilitate local governance." Currently, the values for community boards in df and community districts in ind_df are formatted differently:

In [32]:
df.community_board[0:10]

1499999        08 BRONX
1499998    11 MANHATTAN
1499997       06 QUEENS
1499996       05 QUEENS
1499995     05 BROOKLYN
1499994     05 BROOKLYN
1499993     17 BROOKLYN
1499992    08 MANHATTAN
1499991    04 MANHATTAN
1499990    10 MANHATTAN
Name: community_board, dtype: object

In [33]:
ind_df.cd_short_title[0:10]

0     Manhattan CD 1
1     Manhattan CD 2
2     Manhattan CD 3
3     Manhattan CD 4
4     Manhattan CD 5
5     Manhattan CD 6
6     Manhattan CD 7
7     Manhattan CD 8
8     Manhattan CD 9
9    Manhattan CD 10
Name: cd_short_title, dtype: object

Below, each community district short title is converted to uppercase, 'CD' is removed, and the number is moved to the front of the string, as in the 311 dataset.

In [34]:
#format community district string values so that they match community board vals 

pd.options.mode.chained_assignment = None 

for row in ind_df.index:
    upper = ind_df.at[row, "cd_short_title"].upper().replace(' CD','')
    num_first = "{}".format(upper[-2:] + ' ' + upper[:-2]).strip()
    ind_df.cd_short_title.loc[row] = num_first

The below function removes all 0's from the beginning of single-digit community board numbers in the 311 dataset. The subsequent list comprehension uses the function to update each value in the community board column.

In [35]:
def format_cb(cb_string):

  """Takes in a string value for a community board in the 311 dataframe. If the 
     string starts with '0', updates the string to contain every letter after 0."""  
  
  if cb_string.startswith('0'):
        return cb_string[1:]
  else:
        return cb_string

In [36]:
df.community_board = [format_cb(i) for i in df.community_board]

The community board column contains more unique values than the community district column, as shown below. 

In [37]:
len(df.community_board.unique())

76

In [38]:
len(ind_df.cd_short_title.unique())

59

We'll remove every row without a matching value by first labelling the non-matches and then overwriting the 311 dataset to exclude them.

In [39]:
def label_non_match(cb):

  """Takes in a community board value in the 311 dataset. If the value does not 
     match one of the unique values in the indicators dataset, returns the 
     string 'Non-match.'"""
    
  if cb in ind_df.cd_short_title.unique():
        return cb 
  
  else:
        return 'Non-match'

In [40]:
df.community_board = df.community_board.map(lambda cb: label_non_match(cb))

In [41]:
df = df[df.community_board != 'Non-match']

Now that all of the community district/board values have a match in both columns, we can create a column with a matching title in the indicators dataframe and perform an inner join with the 311 dataframe. 

In [42]:
#create new column with matching name to join data, and drop duplicate column
ind_df['community_board'] = ind_df.cd_short_title
df = pd.merge(df, ind_df, how='inner')

df.drop("cd_short_title", axis=1, inplace=True)

# Exporting the Data

The code below can be used to export the dataframes to Google Drive. Simply change the file paths to paths of your choice.

In [43]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
df.to_csv('/content/gdrive/My Drive/Colab Notebooks/community_board_311.csv', header=True)

# References
[Bar Charts in Python](https://plotly.com/python/bar-charts/)

[Mapbox Density Heatmap in Python](https://plotly.com/python/mapbox-density-heatmaps/)

[Bubble Charts in Python](https://plotly.com/python/bubble-charts/)

[Filled Area Plots in Python](https://plotly.com/python/filled-area-plots/)

[Seaborn Heatmap](https://seaborn.pydata.org/generated/seaborn.heatmap.html)

[Masked Wordcloud](https://amueller.github.io/word_cloud/auto_examples/masked.html)